# Combining templates and plotting systematic shifts

Author(s): Raghav Kansal, Javier Duarte

In [ ]:
import pickle
import json
from pathlib import Path
from HH4b.hh_vars import years

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
templates_path = Path("src/HH4b/postprocessing/templates/24May16")

## Combining templates

In [ ]:
templates = {}
year = "2022-2023"

with (templates_path / f"{year}_templates.pkl").open("rb") as f:
    templates = pickle.load(f)

In [ ]:
templates

## Testing JES plots

In [ ]:
import hist
from hist import Hist
import numpy as np
from HH4b.hh_vars import jecs, jmsr, bg_keys, sig_keys, LUMI
from HH4b import utils, plotting, postprocessing
from HH4b.postprocessing import datacardHelpers, Region
import warnings
from pathlib import Path

In [ ]:
main_plot_dir = Path("plots/PostProcessing/24May16/Templates")


selection_regions = {
    "pass_bin1": Region(
        cuts={
            "Category": [1, 2],
        },
        label="Bin1",
    ),
    "pass_bin2": Region(
        cuts={
            "Category": [2, 3],
        },
        label="Bin2",
    ),
    "pass_bin3": Region(
        cuts={
            "Category": [3, 4],
        },
        label="Bin3",
    ),
    "fail": Region(
        cuts={
            "Category": [4, 5],
        },
        label="Fail",
    ),
}

In [ ]:
def combine_hists(*hists):
    csamples = []
    for h in hists:
        csamples += list(h.axes[0])

    reth = Hist(
        hist.axis.StrCategory(csamples, name="Sample"),
        *hists[0].axes[1:],
        storage="weight",
    )

    for h in hists:
        for sample in h.axes[0]:
            reth.view(flow=True)[utils.get_key_index(reth, sample), ...] = h[sample, ...].view(
                flow=True
            )

    return reth

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))

# this is needed for some reason to update the font size for the first plot
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plt.rcParams.update({"font.size": 24})
plt.close()

In [ ]:
warnings.filterwarnings("ignore")

sig_key = "hh4b"
# sig_key = "vbfhh4b-k2v0"

for rname, region in selection_regions.items():
    plot_dir = main_plot_dir / year / "wshifts"
    plot_dir.mkdir(exist_ok=True, parents=True)
    for wshift in list((jecs).keys()):
        if wshift in jecs:
            # adding jshift-ed histograms into the same histogram
            cjshift_templates = [templates[rname]]
            for shift in ["up", "down"]:
                # new histogram with sample names renamed to "{sample}_{jsf}_{shift}"
                jt = templates[f"{rname}_{wshift}_{shift}"]
                slabels = [f"{s}_{wshift}_{shift}" for s in jt.axes[0]]
                rjt = Hist(
                    hist.axis.StrCategory(slabels, name="Sample"),
                    *jt.axes[1:],
                    storage="weight",
                )
                rjt.view()[...] = jt.view()
                cjshift_templates.append(rjt)

                cjt = combine_hists(*cjshift_templates)
                shift_label = wshift
        else:
            shift_label = postprocessing.weight_shifts[wshift].label
            cjt = templates[rname]
        print(cjt)

        plotting.sigErrRatioPlot(
            cjt,
            sig_key,
            wshift,
            "$m_{reg}^{2}$ (GeV)",
            f"{region.label} Region {shift_label} Variations",
            plot_dir,
            f"{rname}_sig_{wshift}",
            show=True,
        )